In [4]:
# !pip install tensorflow

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)

In [36]:
import os
import pandas as pd


main_folder = "/content/drive/MyDrive/data"

all_patients_data = []

for patient_folder in os.listdir(main_folder):
    patient_path = os.path.join(main_folder, patient_folder)

    if os.path.isdir(patient_path):

        blood_file = os.path.join(patient_path, "blood_monitoring.csv")
        bp_file = os.path.join(patient_path, "bp_monitoring.csv")
        lab_file = os.path.join(patient_path, "lab_results.csv")


        blood_df = pd.read_csv(blood_file)
        bp_df = pd.read_csv(bp_file)
        lab_df = pd.read_csv(lab_file)


        merged_df = blood_df.merge(bp_df, on=["Patient_ID", "Date", "Time"], how="outer")


        lab_columns = set(lab_df.columns) - {"Patient_ID", "Date"}


        existing_columns = lab_columns.intersection(merged_df.columns)


        lab_df = lab_df.drop(columns=existing_columns, errors="ignore")


        merged_df = merged_df.merge(lab_df, on=["Patient_ID", "Date"], how="outer")


        merged_df = merged_df.sort_values(by=["Patient_ID", "Date"], ascending=False)


        merged_df[list(existing_columns)] = merged_df[list(existing_columns)].ffill()


        all_patients_data.append(merged_df)
final_df = pd.concat(all_patients_data, ignore_index=True)
final_df=final_df.drop("Lab Result",axis=1)

final_output_path = "merged_patient_data.csv"
final_df.to_csv(final_output_path, index=False)

print(f"Final merged dataset saved at: {final_output_path}")


Final merged dataset saved at: merged_patient_data.csv


In [37]:
df=pd.read_csv("/content/merged_patient_data.csv")

In [32]:
df.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,125.3,95.4,1.18,67.0,P12346,95/84,62,15,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
1,28-02-2025,0.17.33,93.9,99.3,1.15,62.9,P12346,95/83,89,13,36.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
2,28-02-2025,0.27.33,185.0,97.2,0.89,59.4,P12346,122/67,69,17,37.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
3,28-02-2025,0.37.33,108.1,98.1,0.93,70.2,P12346,140/62,76,13,37.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
4,28-02-2025,0.47.33,127.6,97.3,0.89,55.9,P12346,135/80,60,19,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11


In [38]:
df.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [39]:
# preprocessing data

df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H.%M.%S')
df = df.drop(columns=['Date', 'Time'])
df = df.sort_values(by='Timestamp').reset_index(drop=True)
df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)
df = df.drop(columns=['Blood Pressure'])
df = df.fillna(method='ffill')



scaler = MinMaxScaler()
def normalize_group(group):
    df_numeric = group.drop(columns=['Patient_ID', 'Timestamp'])
    group[df_numeric.columns] = scaler.fit_transform(df_numeric)
    return group

df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)


def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

predictions = {}
for patient_id, group in df.groupby('Patient_ID'):
    data_values = group.drop(columns=['Patient_ID', 'Timestamp']).values


    X, y = create_sequences(data_values, seq_length=5)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = Sequential([
        LSTM(50, activation='relu', return_sequences=True, input_shape=(5, X.shape[2])),
        LSTM(50, activation='relu'),
        Dense(X.shape[2])
    ])


    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

    last_sequence = X[-1].reshape(1, 5, X.shape[2])
    next_row_prediction = model.predict(last_sequence)
    next_row_original = scaler.inverse_transform(next_row_prediction)


    predictions[patient_id] = np.round(next_row_original, 2)

for patient_id, pred in predictions.items():
    print(f"Predicted Next Row for Patient {patient_id}:")
    print(np.round(pred, 2))

Epoch 1/20


<ipython-input-39-71857404e481>:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
<ipython-input-39-71857404e481>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Patient_ID', group_keys=False).apply(normalize_group)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.2705 - val_loss: 0.1092
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0715 - val_loss: 0.1147
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0505 - val_loss: 0.1052
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0438 - val_loss: 0.1069
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0434 - val_loss: 0.1102
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0414 - val_loss: 0.1076
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0422 - val_loss: 0.1124
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0425 - val_loss: 0.1104
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0418 - val_loss: 0.1112
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0411 - val_loss: 0.1103
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0407 - val_loss: 0.1098
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0400 - val_loss: 0.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.2352 - val_loss: 0.1877
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0573 - val_loss: 0.1846
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0417 - val_loss: 0.1854
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0408 - val_loss: 0.1922
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0398 - val_loss: 0.1891
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0391 - val_loss: 0.1860
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0391 - val_loss: 0.1844
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0386 - val_loss: 0.1825
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0392 - val_loss: 0.1818
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0384 - val_loss: 0.1857
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0381 - val_loss: 0.1822
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0390 - val_loss: 0.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.2307 - val_loss: 0.1710
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0767 - val_loss: 0.1428
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0519 - val_loss: 0.1535
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0452 - val_loss: 0.1529
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0419 - val_loss: 0.1541
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0419 - val_loss: 0.1539
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0408 - val_loss: 0.1539
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0409 - val_loss: 0.1567
Epoch 9/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0410 - val_loss: 0.1555
Epoch 10/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0406 - val_loss: 0.1601
Epoch 11/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0404 - val_loss: 0.1627
Epoch 12/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0401 - val_loss: 0.16